# Executive Budgeting + Forecasting — Forecast Accuracy (Monthly)
**Provided Forecast vs Seasonal-Naive baseline**  
*Project MVP: quantify how the current forecast performs against a naive yardstick.*

---

## Parameters
- **TEST_YEAR:** `2024` 
- **GRAIN:** Monthly
- **GROUP BY:** Department
- **BASELINE:** Seasonal-Naive (`ŷ_t = A_{t-12}` within each group)

---

## Objective
- Compute portfolio-ready accuracy metrics for **TEST_YEAR** by **Department**.
- Benchmark the **Provided Forecast** against a **Seasonal-Naive** baseline.
- Produce tidy outputs for dashboards and the README.

---

## Inputs
- `data/processed/monthly_sim.csv`

**Required columns**
- `Month` (date or parseable string, month start preferred)
- `Year` (int)
- `Department` (str)
- `Actual` (float)
- `Forecast` (float)  <!-- Provided forecast from 3 month rolling  mean-->
- `Budget` (float), `ShockFlag` (bool/int)

---

## Metrics (percent)
- **MAPE:** mean(|A−F| / |A|) × 100  *(skip terms where A = 0)*
- **WAPE:** sum(|A−F|) / sum(|A|) × 100
- **sMAPE:** mean( |A−F| / ((|A|+|F|)/2) ) × 100  *(skip terms where denom = 0)*

---

## Method
1. Load monthly data; enforce one row per `(Department, Month)`.
2. Create **Naive** = `Actual.shift(12)` within each `Department`.
3. Filter to `Year == TEST_YEAR`.
4. For each `Department`, compute MAPE/WAPE/sMAPE for:
   - **Provided**: `Forecast` vs `Actual`
   - **Naive**: `Naive` vs `Actual`
5. Save results and a simple comparison chart.

---

## Outputs
- `outputs/metrics/accuracy_{TEST_YEAR}_by_department.csv`
- `outputs/figures/accuracy_wape_{TEST_YEAR}.png`

---

## Acceptance Checklist
- [ ] 12 months present for every department in **TEST_YEAR**  
- [ ] `Naive[t] == Actual[t−12]` (spot-check a few months per dept)  
- [ ] Metric functions pass a toy sanity test (e.g., A=[100,100], F=[90,110] → ~10%)  
- [ ] CSV written with Provided vs Naive side-by-side  
- [ ] One-sentence executive takeaway added to README/dashboard

---

> **Notes**
> - If `ShockFlag` exists, consider a second table isolating shock vs non-shock months.
> - Prefer year-tagged **outputs**, but keep the notebook name generic with a `TEST_YEAR` parameter.

In [ ]:
# --- Imports ---
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# --- Parameters ---
TEST_YEAR = 2024
MONTHLY_PATH = Path("data/monthly_sim.csv") # Update path as needed, set to my local path

# --- Load ---
assert MONTHLY_PATH.exists(), f"File not found: {MONTHLY_PATH}"
df = pd.read_csv(MONTHLY_PATH, parse_dates=["Month"])

# --- Category Check and Dtype Check ---
expected = ["Month", "Year", "Department", "Actual", "Forecast"]
missing = [c for c in expected if c not in df.columns]
assert not missing, f"Missing required columns: {missing}"

df["Year"] = df["Year"].astype(int)
df["Department"] = df["Department"].astype(str)

# Sort for later ETL and groupings
df = df.sort_values(["Department", "Month"]).reset_index(drop=True)

# Quick Peak at the Data
display(df.head(3))
print(df.dtypes)